> From the PO.DAAC Cookbook, to access the GitHub version of the notebook, follow [this link](insert link to notebook).

# Hydrocron API: SWOT Time Series Examples

#### *Authors: Nikki Tebaldi and Cassandra Nickles, NASA PO.DAAC*

### Summary:

[Hydrocron](https://podaac.github.io/hydrocron/intro.html) is an API that repackages the `SWOT_L2_HR_RIVERSP_2.0` dataset into csv or geojson formats that make time series analysis easier. This notebook will highlight how to utilize hydrocron and convert its output into a readable geodatabase of data from multiple SWOT reaches identified from the [SWORD Database](https://www.swordexplorer.com/).

### Requirements:
Any compute environment, local or the cloud.

### Learning Objectives:
- Obtain a list of SWORD IDs for a region of interest
- Access SWOT river vector product attributes for multiple reach IDs via the Hydrocron API
- Convert accessed time series data into readable database
- Plot one time series variable from multiple reaches

### Cite the Hydrocron tool via the following:
 `

## Import Packages

In [1]:
import requests
import pprint

## Obtain SWORD IDs for a river

In this section, we query the Feature Translation Service (FTS) to give us a list of river IDs from the SWOT River Database [(SWORD)](https://www.swordexplorer.com/). One river reach ID has the format `CBBBBBRRRRT` and a node ID has the format, `CBBBBBRRRRNNNT` where `C` stands for continent, `B` for basin, `R` for reach, `N` for node, and `T` for type. The first 6 digits of the id (CBBBBB) are the [HydroBASINS](https://www.hydrosheds.org/products/hydrobasins) Pfafstetter level 6 basin code, where the first digit represents one of nine continental regions (1 = Africa, 2 = Europe, 3 = Siberia, 4 = Asia, 5 = Oceania, 6 = South America, 7 = North America, 8 = Arctic, 9 = Greenland), and the remaining digits are nested basin levels 2–6. We recommend looking for your region of interest in the SWORD database and identifying the code for your basin/river of interest.

In this example, we use ID `732520`. This ID represents multiple reaches along the Savannah River and its tributaries in Georgia, USA. Note that this ID includes every reach included in the broader category, for example, individual reach `73252000161`. 

In [21]:
Savannah_reaches = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/reach/732520")

pprint.pprint(Savannah_reaches.json(), compact=True, width=60, depth=2)

{'hits': 192,
 'results': {'73252000013': {...},
             '73252000043': {...},
             '73252000051': {...},
             '73252000061': {...},
             '73252000071': {...},
             '73252000081': {...},
             '73252000091': {...},
             '73252000101': {...},
             '73252000111': {...},
             '73252000121': {...},
             '73252000131': {...},
             '73252000141': {...},
             '73252000151': {...},
             '73252000161': {...},
             '73252000171': {...},
             '73252000181': {...},
             '73252000191': {...},
             '73252000201': {...},
             '73252000211': {...},
             '73252000221': {...},
             '73252000231': {...},
             '73252000244': {...},
             '73252000251': {...},
             '73252000261': {...},
             '73252000271': {...},
             '73252000284': {...},
             '73252000291': {...},
             '73252000303': {...},
      

In [35]:
# This is only a partial result from page 1, how do we also append from the rest of the pages in the response?
IDs = list(Savannah_reaches.json()['results'].keys())